In [1]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GenerationConfig, GPT2Config
import torch
import torch.nn as nn
import os
from pathlib import Path

import transformers
from onnxruntime import InferenceSession
from transformers.onnx.features import FeaturesManager
from optimum.exporters.tasks import TasksManager

In [2]:
import numpy as np

def save_to(x, fp):
    os.makedirs(os.path.dirname(fp), exist_ok=True)
    print(x.dtype)
    np_x = x.detach().numpy()
    with open(fp, 'wb') as f:
        np_x.tofile(f)

def save_past_key_values(x, n, d):
    for l in range(n):
        k, v = x[l]
        fp = os.path.join(d, "past_key_values", "{}.{}".format('past', l))
        save_to(k, fp + '.key')
        save_to(v, fp + '.value')

In [3]:
torch.manual_seed(2333)
# torch.set_num_threads(1)

In [4]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval()
# model.config.return_dict = False
text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='pt')
encoded_input

{'input_ids': tensor([[3041, 5372,  502,  416,  597, 2420,  345, 1549,  588,   13]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [5]:
print(model)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)


In [6]:
# Stage 0
encoded_input_0 = {
    'input_ids': torch.randint(0, 50257, (64, 256)),
    'past_key_values': None
}


In [7]:
%%time
with torch.no_grad():
    output = model(**encoded_input_0)
print(output.logits.shape)
print(output.past_key_values[0][0].size())

torch.Size([64, 256, 50257])
torch.Size([64, 12, 256, 64])
CPU times: user 2min 24s, sys: 13.5 s, total: 2min 37s
Wall time: 10.1 s


In [8]:
# prepare for stage 1
next_input = torch.argmax(output.logits[..., -1, :], dim=-1, keepdim=True)
next_input.size()

torch.Size([64, 1])

In [11]:
%%time

# Stage 1
model.config.use_cache = False
with torch.no_grad():
    next_output = model(next_input, past_key_values=output.past_key_values)
print(next_output)

CausalLMOutputWithCrossAttentions(loss=None, logits=tensor([[[-58.6922, -59.2055, -59.6426,  ..., -59.9432, -61.9034, -57.4000]],

        [[-64.6238, -63.8287, -63.8751,  ..., -71.0728, -69.9986, -64.9882]],

        [[-55.1815, -55.1395, -56.2996,  ..., -58.6072, -57.5121, -54.7201]],

        ...,

        [[-78.0860, -77.6450, -77.4851,  ..., -86.3432, -84.5547, -78.6800]],

        [[-57.6484, -57.4646, -58.4835,  ..., -60.0685, -60.9240, -57.5087]],

        [[-62.8545, -63.7144, -64.6444,  ..., -66.4942, -67.8090, -63.4077]]]), past_key_values=None, hidden_states=None, attentions=None, cross_attentions=None)
CPU times: user 3.03 s, sys: 562 ms, total: 3.59 s
Wall time: 228 ms


In [9]:
%%time

# Stage 1
with torch.no_grad():
    next_output = model(next_input, past_key_values=output.past_key_values)
print(next_output.logits.shape)
print(next_output.past_key_values[0][0].size())

torch.Size([64, 1, 50257])
torch.Size([64, 12, 257, 64])
CPU times: user 2.32 s, sys: 1.68 s, total: 4 s
Wall time: 257 ms


In [14]:
# Export
def past_key_values_names(n: int, prefix: str):
    for i in range(n):
        yield "{}.{}.key".format(prefix, i)
        yield "{}.{}.value".format(prefix, i)

def prepare_dimension(in_name, out_name):
    ds = {}
    for name in in_name + out_name:
        if name.startswith('past') or name.startswith('present'):
            ds[name] = {2: 'seq_length'}
    return ds

input_names = ["input_ids"] + list(past_key_values_names(12, "past"))
# output_names = ["logits"] + list(past_key_values_names(12, "present"))
output_names = ["logits"]
# dyn_dimensions = prepare_dimension(input_names, output_names)
print(input_names)
print(output_names)
# print(dyn_dimensions)

dummy_input = {
    'input_ids': next_input,
    'past_key_values': output.past_key_values
}

['input_ids', 'past.0.key', 'past.0.value', 'past.1.key', 'past.1.value', 'past.2.key', 'past.2.value', 'past.3.key', 'past.3.value', 'past.4.key', 'past.4.value', 'past.5.key', 'past.5.value', 'past.6.key', 'past.6.value', 'past.7.key', 'past.7.value', 'past.8.key', 'past.8.value', 'past.9.key', 'past.9.value', 'past.10.key', 'past.10.value', 'past.11.key', 'past.11.value']
['logits']


In [15]:
torch.onnx.export(model,
                  dummy_input,
                  "./model_onnx/gpt2.onnx",
                  verbose=False,
                  input_names=input_names,
                  output_names=output_names,
                  dynamic_axes=None)

============== Diagnostic Run torch.onnx.export version 2.0.0+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [16]:
save_to(dummy_input['input_ids'], './model_onnx/input_ids.dat')
save_past_key_values(dummy_input['past_key_values'], 12, './model_onnx')

torch.int64
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32
torch.float32


In [19]:
gen_cfg = GenerationConfig(
    pad_token_id = tokenizer.eos_token_id, 
    use_cache = True,
    max_new_tokens = 128)

In [20]:
%%time
output = model.generate(**encoded_input, generation_config=gen_cfg)
print(tokenizer.decode(output[0]))

Replace me by any text you'd like.

I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you're aware of this, but I'm not sure if you
CPU times: user 5.14 s, sys: 264 ms, total: 5.41 s
Wall time: 5.46 s
